In [2]:
#!pip install xvec
# using xvec https://xvec.readthedocs.io/en/stable/zonal_stats.html

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dask.dataframe as dd
import seaborn as sns
import xarray as xr
import geopandas as gpd
import xvec
import coiled
import dask
from global_snowmelt_runoff_onset.config import Config

/home/eric/.local/lib/python3.10/site-packages/paramiko/pkey.py:100: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
/home/eric/.local/lib/python3.10/site-packages/paramiko/transport.py:259: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


In [2]:
config = Config('../config/global_config.txt')

Configuration loaded:
resolution = 0.00072000072000072
spatial_chunk_dim = 2048
bbox_left = -179.999
bbox_right = 179.999
bbox_top = 81.099
bbox_bottom = -59.999
wy_start = 2015
wy_end = 2024
low_backscatter_threshold = 0.001
min_monthly_acquisitions = 2
max_allowed_days_gap_per_orbit = 30
min_years_for_median_std = 3
valid_tiles_geojson_path = ../processing/valid_tiles.geojson
tile_results_path = ../processing/tile_results.csv
global_runoff_zarr_store_azure_path = snowmelt/snowmelt_runoff_onset/global.zarr
seasonal_snow_mask_zarr_store_azure_path = snowmelt/snow_mask_v2/global_modis_snow_mask.zarr


In [3]:
global_ds = xr.open_zarr(config.global_runoff_store, consolidated=True,decode_coords='all')
global_ds

<xarray.Dataset> Size: 5TB
Dimensions:              (latitude: 195970, longitude: 499998, water_year: 10)
Coordinates:
  * latitude             (latitude) float64 2MB 81.1 81.1 81.1 ... -60.0 -60.0
  * longitude            (longitude) float64 4MB -180.0 -180.0 ... 180.0 180.0
    spatial_ref          int32 4B ...
  * water_year           (water_year) int64 80B 2015 2016 2017 ... 2023 2024
Data variables:
    runoff_onset         (water_year, latitude, longitude) float32 4TB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    runoff_onset_median  (latitude, longitude) float32 392GB dask.array<chunksize=(2048, 2048), meta=np.ndarray>
    runoff_onset_std     (latitude, longitude) float32 392GB dask.array<chunksize=(2048, 2048), meta=np.ndarray>
Attributes:
    processed_tiles:  []

In [4]:
url = (f"https://data.earthenv.org/mountains/standard/GMBA_Inventory_v2.0_standard_300.zip")
gmba_gdf = gpd.read_file("zip+" + url)
gmba_gdf

,GMBA_V2_ID,GMBA_V1_ID,MapName,WikiDataUR,MapUnit,Hier_Lvl,Feature,Area,Perimeter,Elev_Low,...,Name_ES,Name_FR,Name_PT,Name_RU,Name_ZH,LocalNames,ColorAll,ColorBasic,Color300,geometry
0,19102,None,Arctic Ocean,https://www.wikidata.org/wiki/Q788,Aggregated,1,Ocean,57205.24576,22872.03994,0.0,...,Océano Ártico,Océan Arctique,Oceano Ártico,Северный Ледовитый океан,北冰洋,Kadagatang Artiko (Norwegian),1,0,1,"MULTIPOLYGON (((-8.63071 70.96548, -8.62822 70..."
1,11134,3.04.01,Appalachian Mountains,https://www.wikidata.org/wiki/Q93332,Aggregated,2,Mountain range with well-recognized name,129875.02600,37431.11230,-159.0,...,Apalaches,Appalaches,Apalaches,Аппалачи,阿巴拉契亚山脉,None,4,0,1,"MULTIPOLYGON (((-85.78819 33.4368, -85.80703 3..."
2,12390,None,Cordillera Centroamericana,https://www.wikidata.org/wiki/Q5788379,Aggregated,2,Mountain range with well-recognized name,284121.62390,31599.87888,-1.0,...,Cordillera Centroamericana,None,None,None,None,None,1,0,2,"MULTIPOLYGON (((-77.41295 5.56954, -77.41741 5..."
3,12497,None,Caribbean Islands,https://www.wikidata.org/wiki/Q664609,Aggregated,2,Archipelago,61738.67557,10329.79926,-40.0,...,Caribe,Caraïbes,Caraíbas,Карибы,加勒比地区,None,4,0,4,"MULTIPOLYGON (((-61.05987 10.83557, -61.05311 ..."
4,14271,None,South Atlantic Islands,https://www.wikidata.org/wiki/Q1482804,Aggregated,2,Geographically-defined sub-range,5579.35771,1487.00483,-133.0,...,Océano Atlántico Sur,Océan Atlantique Sud,Atlântico Sul,Южная Атлантика,南大西洋,None,3,0,2,"MULTIPOLYGON (((3.39049 -54.395, 3.41145 -54.4..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286,11208,5.08.19,Central Range,None,Aggregated,6,Mountain range with well-recognized name,263615.62150,12354.30388,0.0,...,Cordillera Central,Chaîne Centrale,None,Центральный хребет Новой Гвинеи,巴布亞新幾內亞高地,None,6,0,4,"MULTIPOLYGON (((150.33845 -10.40741, 150.34297..."
287,12748,None,Borneo,https://www.wikidata.org/wiki/Q36117,Aggregated,6,Island,248480.11360,28667.97285,-48.0,...,Isla de Borneo,Bornéo,Bornéu,Калимантан,婆罗洲,Kalimantan (Acehnese); Nusa Kalimantan (Baline...,6,0,5,"MULTIPOLYGON (((114.97609 -3.51973, 114.9728 -..."
288,12756,None,Java,https://www.wikidata.org/wiki/Q3757,Aggregated,6,Island,42186.46494,6852.31987,0.0,...,Java,Java,Java,Ява,爪哇岛,Jawa (Acehnese); Jawa (Balinese); Jawa (Banjar...,1,0,4,"MULTIPOLYGON (((114.06353 -8.62899, 114.05197 ..."
289,18917,None,Sulawesi Archipelago,None,Aggregated,6,Archipelago,128619.86330,11614.45005,-16.0,...,None,None,None,None,None,None,1,0,1,"MULTIPOLYGON (((120.73094 -7.09701, 120.71339 ..."


In [6]:
aggregated = global_ds.xvec.zonal_stats(gmba_gdf.geometry, x_coords='latitude', y_coords='longitude', stats=['mean'], dask=True)

MemoryError: Unable to allocate 730. GiB for an array with shape (499998, 195970) and data type float64

In [ ]:
# https://discourse.pangeo.io/t/advice-for-scalable-raster-vector-extraction/4129/18
# 
from exactextract import exact_extract
from osgeo import ogr, gdal

gdal.UseExceptions()
ogr.UseExceptions()

rast = gdal.Open("/home/dan/data/hfp_2021_100m_v1-2_cog.tif")

padus_fname = "/home/dan/data/PADUS3_0Geopackage.gpkg"

vrt = f"""
<OGRVRTDataSource>
    <OGRVRTWarpedLayer>
        <OGRVRTLayer name="combined">
            <SrcDataSource>{padus_fname}</SrcDataSource>
            <SrcLayer>PADUS3_0Combined_DOD_TRIB_Fee_Designation_Easement</SrcLayer>
        </OGRVRTLayer>
        <TargetSRS>{rast.GetSpatialRef().ExportToWkt()}</TargetSRS>
    </OGRVRTWarpedLayer>
</OGRVRTDataSource>
"""

polys = ogr.Open(vrt)

exact_extract(rast, polys, 'mean', output='gdal', output_options = {'filename' : '/tmp/out.csv' })

In [ ]:
# other options
# flox + rasterizing https://discourse.pangeo.io/t/advice-for-scalable-raster-vector-extraction/4129/18
# https://regionmask.readthedocs.io/en/stable/ https://xarray.dev/blog/flox